## 📦 Installation

First, let's install the required dependencies.

In [ ]:
# Install Google ADK and dependencies
!pip install -q google-adk python-dotenv

## 🔑 API Key Setup

You need a Google AI Studio API key. Get one free at: https://aistudio.google.com/apikey

In [ ]:
import os
from kaggle_secrets import UserSecretsClient

# Get API key from Kaggle Secrets
# Go to Add-ons > Secrets > Add new secret named GOOGLE_API_KEY
try:
    user_secrets = UserSecretsClient()
    os.environ['GOOGLE_API_KEY'] = user_secrets.get_secret('GOOGLE_API_KEY')
    print("✅ API Key loaded from Kaggle Secrets!")
except Exception as e:
    print(f"❌ Failed to load API key: {e}")
    print("Please add GOOGLE_API_KEY in Add-ons > Secrets")

# Use Google AI Studio (not Vertex AI)
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'False'

print("✅ Environment configured!" if os.environ.get('GOOGLE_API_KEY') else "❌ API Key missing!")

## 🏗️ Architecture

InfoShield uses a **Sequential Multi-Agent** architecture:

```
User Query → Analyzer Agent → Search Agent → Credibility Agent → Synthesizer → Response
```

Each agent has a specific role:
- **Analyzer**: Extracts sentiment, urgency, location, disaster type
- **Search**: Finds real-time news and official sources
- **Credibility**: Scores the reliability of information
- **Synthesizer**: Creates the final verification report

## ⚙️ Configuration

In [ ]:
# Configuration constants
MODEL_ID = "gemini-2.5-pro"
MODEL_ID_FAST = "gemini-2.5-flash"

URGENCY_THRESHOLD = 8      # Score >= 8 triggers immediate response
CREDIBILITY_THRESHOLD = 60 # Score >= 60% enables automated response

# Disaster Keywords for Detection
DISASTER_KEYWORDS = [
    "flood", "flooding", "earthquake", "tsunami", "cyclone", "hurricane",
    "tornado", "wildfire", "fire", "landslide", "avalanche", "drought",
    "volcano", "eruption", "storm", "typhoon", "blizzard", "heatwave",
    "rescue", "emergency", "evacuation", "trapped", "help", "sos"
]

# Official Sources for Credibility Scoring
OFFICIAL_SOURCES = [
    # Global
    "un ocha", "red cross", "red crescent", "who", "unicef",
    # News
    "reuters", "ap news", "afp", "bbc", "cnn", "al jazeera",
    # Weather
    "accuweather", "weather.com",
    # India
    "ndrf", "ndma", "imd", "india met",
    # USA
    "fema", "nws", "noaa", "usgs",
    # UK
    "met office", "bbc weather",
    # Japan
    "jma", "japan meteorological agency"
]

print(f"✅ Configuration loaded!")
print(f"   Model: {MODEL_ID}")
print(f"   Urgency Threshold: {URGENCY_THRESHOLD}")
print(f"   Credibility Threshold: {CREDIBILITY_THRESHOLD}%")

## 🛠️ Tools Implementation

### Tool 1: Query Analyzer

In [ ]:
import re
from typing import Dict, Any


def analyze_query(query: str) -> Dict[str, Any]:
    """
    Analyze a disaster-related query for sentiment, urgency, and location.

    Args:
        query: The user's disaster-related query string.

    Returns:
        Dictionary with sentiment, urgency_score, location, disaster_type,
        is_emergency, and keywords_found.
    """
    query_lower = query.lower()

    # Detect disaster keywords
    keywords_found = [kw for kw in DISASTER_KEYWORDS if kw in query_lower]

    # Urgency indicators
    panic_indicators = ["help", "sos", "emergency", "trapped", "dying", "drowning"]
    urgent_indicators = ["now", "immediately", "urgent", "quickly", "asap", "!"]

    # Calculate urgency score
    urgency_score = 3  # Base score
    if any(ind in query_lower for ind in panic_indicators):
        urgency_score += 4
    if any(ind in query_lower for ind in urgent_indicators):
        urgency_score += 2
    if query.count("!") >= 2:
        urgency_score += 1
    if len(keywords_found) > 0:
        urgency_score += len(keywords_found)
    urgency_score = min(10, max(1, urgency_score))

    # Determine sentiment
    if urgency_score >= 8:
        sentiment = "panic"
    elif urgency_score >= 6:
        sentiment = "urgent"
    elif urgency_score >= 4:
        sentiment = "concerned"
    elif "?" in query and len(keywords_found) == 0:
        sentiment = "curious"
    else:
        sentiment = "neutral"

    # Extract location
    location = "Unknown"
    location_patterns = [
        r"in\s+([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)",
        r"at\s+([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)",
        r"near\s+([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)",
        r"from\s+([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)",
    ]
    for pattern in location_patterns:
        match = re.search(pattern, query)
        if match:
            location = match.group(1)
            break

    # Identify disaster type
    disaster_type = None
    disaster_mapping = {
        "flood": ["flood", "flooding", "water entering"],
        "earthquake": ["earthquake", "quake", "tremor", "seismic"],
        "tsunami": ["tsunami", "tidal wave"],
        "cyclone": ["cyclone", "hurricane", "typhoon", "storm"],
        "fire": ["fire", "wildfire", "blaze", "burning"],
        "landslide": ["landslide", "mudslide", "debris"],
    }
    for dtype, keywords in disaster_mapping.items():
        if any(kw in query_lower for kw in keywords):
            disaster_type = dtype
            break

    is_emergency = urgency_score >= 8 or any(ind in query_lower for ind in panic_indicators)

    return {
        "sentiment": sentiment,
        "urgency_score": urgency_score,
        "location": location,
        "disaster_type": disaster_type,
        "is_emergency": is_emergency,
        "keywords_found": keywords_found
    }


# Test the analyzer
test_query = "Help! There's flooding in Mumbai, water is entering houses!"
result = analyze_query(test_query)
print(f"Query: {test_query}")
print(f"Analysis: {result}")

### Tool 2: Credibility Scorer

In [ ]:
def calculate_credibility(
    search_summary: str,
    sources_mentioned: str = "",
    location: str = "",
    disaster_type: str = ""
) -> Dict[str, Any]:
    """
    Calculate credibility score based on search results and sources.

    Args:
        search_summary: Summary of search results.
        sources_mentioned: Comma-separated list of sources.
        location: Location from the query.
        disaster_type: Type of disaster identified.

    Returns:
        Dictionary with score, reasoning, sources_found, etc.
    """
    search_lower = search_summary.lower()
    sources_lower = sources_mentioned.lower()

    score = 0
    reasons = []
    sources_found = []
    official_count = 0
    news_count = 0

    # Check for official sources
    for source in OFFICIAL_SOURCES:
        if source in search_lower or source in sources_lower:
            sources_found.append(source)
            if source in ["ndrf", "ndma", "fema", "government", "ministry", "official"]:
                official_count += 1
            else:
                news_count += 1

    # Score official sources (max 40)
    if official_count > 0:
        official_points = min(40, official_count * 20)
        score += official_points
        reasons.append(f"Found {official_count} official source(s) (+{official_points})")

    # Score news sources (max 30)
    if news_count > 0:
        news_points = min(30, news_count * 10)
        score += news_points
        reasons.append(f"Found {news_count} news source(s) (+{news_points})")

    # Recency indicators (+10)
    recency_keywords = ["today", "now", "just", "breaking", "latest", "current"]
    if any(kw in search_lower for kw in recency_keywords):
        score += 10
        reasons.append("Recent/current event indicators (+10)")

    # Location specificity (+10)
    if location and location.lower() != "unknown":
        if location.lower() in search_lower:
            score += 10
            reasons.append(f"Location '{location}' confirmed (+10)")

    # Multiple source corroboration (+10)
    if len(sources_found) >= 3:
        score += 10
        reasons.append("Multiple sources corroborate (+10)")

    # Clamp score to 0-100
    score = min(100, max(0, score))

    # Determine status
    if score >= 70:
        status = "Verified"
    elif score >= 50:
        status = "Partially Verified"
    elif score >= 30:
        status = "Unverified"
    else:
        status = "Likely Misinformation"

    return {
        "score": score,
        "status": status,
        "reasoning": "; ".join(reasons) if reasons else "No credible sources found",
        "sources_found": sources_found,
        "official_sources_count": official_count,
        "news_sources_count": news_count
    }


# Test the credibility scorer
test_summary = "NDRF confirms flooding in Chennai. Reuters reports heavy rain. BBC coverage ongoing."
result = calculate_credibility(test_summary, "ndrf.gov.in, reuters.com", "Chennai", "flood")
print(f"Credibility Score: {result['score']}/100")
print(f"Status: {result['status']}")
print(f"Reasoning: {result['reasoning']}")

### Tool 3: Human Review Queue

In [ ]:
import uuid
from datetime import datetime

# In-memory storage for Kaggle (no file system persistence)
HUMAN_REVIEW_QUEUE = []


def save_for_human_review(
    query: str,
    location: str,
    urgency_score: int,
    credibility_score: int
) -> Dict[str, Any]:
    """
    Save a query for human expert verification.

    Args:
        query: The original user query.
        location: Extracted location.
        urgency_score: Urgency score (1-10).
        credibility_score: Credibility score (0-100).

    Returns:
        Dictionary with session_id, status, and estimated_review_time.
    """
    session_id = str(uuid.uuid4())[:8].upper()
    timestamp = datetime.now().isoformat()

    entry = {
        "session_id": session_id,
        "timestamp": timestamp,
        "query": query,
        "location": location,
        "urgency_score": urgency_score,
        "credibility_score": credibility_score,
        "status": "pending"
    }

    HUMAN_REVIEW_QUEUE.append(entry)

    # Estimate review time based on urgency
    if urgency_score >= 8:
        review_time = "15-30 minutes (Priority)"
    elif urgency_score >= 5:
        review_time = "1-2 hours"
    else:
        review_time = "4-8 hours"

    return {
        "session_id": session_id,
        "status": "saved",
        "message": f"Query saved for human review with ID: {session_id}",
        "estimated_review_time": review_time
    }


# Test
result = save_for_human_review(
    "Aliens attacking downtown!",
    "Unknown",
    5,
    15
)
print(f"Human Review: {result}")

## 🤖 Agent Definitions

Now let's create the multi-agent system using Google ADK.

In [ ]:
from datetime import datetime
from google.adk.agents import Agent, SequentialAgent
from google.adk.tools import google_search, FunctionTool

CURRENT_DATE = datetime.now().strftime("%B %d, %Y")

# ============== ANALYZER AGENT ==============
ANALYZER_INSTRUCTION = """You are the Analyzer Agent for InfoShield AI.
Your job is to analyze disaster-related queries using the `analyze_query` tool.

## YOUR ROLE
1. Receive a query.
2. Call the `analyze_query` tool with the query string.
3. Return the analysis results as JSON.

## OUTPUT FORMAT
Return ONLY the tool output as a JSON string.
"""

def create_analyzer_agent():
    analysis_tool = FunctionTool(analyze_query)
    return Agent(
        name="analyzer_agent",
        model=MODEL_ID_FAST,
        description="Analyzes disaster queries for urgency, sentiment, and location.",
        instruction=ANALYZER_INSTRUCTION,
        tools=[analysis_tool],
        output_key="analysis_result"
    )


# ============== SEARCH AGENT ==============
SEARCH_INSTRUCTION = f"""You are the Search Agent for InfoShield AI.
Your job is to search for CURRENT, REAL-TIME disaster information.

**Today's date: {CURRENT_DATE}**

## YOUR ROLE
You have access to analysis results in {{analysis_result}}.
Extract location and disaster_type, then perform targeted searches.

## SEARCH STRATEGY
1. "[location] [disaster_type] news today {CURRENT_DATE}"
2. "[location] disaster alert warning official"
3. "[location] weather conditions current"

## OUTPUT FORMAT
Return search findings with source names and dates.
"""

def create_search_agent():
    return Agent(
        name="search_agent",
        model=MODEL_ID,
        description="Searches for real-time disaster information.",
        instruction=SEARCH_INSTRUCTION,
        tools=[google_search],
        output_key="search_result"
    )


# ============== CREDIBILITY AGENT ==============
CREDIBILITY_INSTRUCTION = """You are the Credibility Agent for InfoShield AI.
Your job is to evaluate the trustworthiness of disaster reports.

## YOUR ROLE
1. Access search results in {search_result}.
2. Call `calculate_credibility` with appropriate parameters.
3. Return the credibility assessment as JSON.
"""

def create_credibility_agent():
    credibility_tool = FunctionTool(calculate_credibility)
    return Agent(
        name="credibility_agent",
        model=MODEL_ID,
        description="Evaluates credibility of disaster reports.",
        instruction=CREDIBILITY_INSTRUCTION,
        tools=[credibility_tool],
        output_key="credibility_result"
    )


# ============== SYNTHESIZER AGENT ==============
SYNTHESIZER_INSTRUCTION = f"""You are the InfoShield AI Report Synthesizer.

**Today's Date: {CURRENT_DATE}**

## YOUR ROLE
Create a comprehensive verification report using data from:
- {{analysis_result}}: Sentiment, urgency, location, disaster_type
- {{search_result}}: Search findings with sources
- {{credibility_result}}: Credibility score and status

## FINAL RESPONSE FORMAT

**📊 INFOSHIELD VERIFICATION REPORT**

| Metric | Value |
|--------|-------|
| Status | [status] |
| Credibility Score | [score]/100 |
| Urgency Level | [Low/Medium/High/Critical] |
| Location | [location] |
| Disaster Type | [type] |

**📋 Summary:**
[Bullet point summary of findings]

**🛡️ Safety Advice:**
[Appropriate safety advice]

**📰 Sources:**
[List of sources]

**⚠️ Disclaimer:**
For emergencies, contact local emergency services (911, 112) directly.
"""


# ============== ORCHESTRATOR ==============
def create_orchestrator():
    """Create the sequential multi-agent orchestrator."""
    analyzer = create_analyzer_agent()
    searcher = create_search_agent()
    credibility = create_credibility_agent()

    synthesizer = Agent(
        name="synthesizer_agent",
        model=MODEL_ID,
        description="Creates the final verification report.",
        instruction=SYNTHESIZER_INSTRUCTION,
    )

    return SequentialAgent(
        name="infoshield_orchestrator",
        description="InfoShield AI multi-agent disaster verification system",
        sub_agents=[analyzer, searcher, credibility, synthesizer]
    )


print("✅ Agents defined successfully!")

## 🚀 Running InfoShield AI

Now let's create a runner and test the system!

In [ ]:
import asyncio
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types


async def run_infoshield(query: str) -> str:
    """
    Process a disaster query through InfoShield AI.

    Args:
        query: The user's disaster-related query.

    Returns:
        The verification report as a string.
    """
    print(f"\n🔍 Processing: {query}")
    print("=" * 60)

    # Create orchestrator and runner
    orchestrator = create_orchestrator()
    session_service = InMemorySessionService()

    runner = Runner(
        agent=orchestrator,
        app_name="infoshield_ai",
        session_service=session_service
    )

    # Create session
    session = await session_service.create_session(
        app_name="infoshield_ai",
        user_id="kaggle_user"
    )

    # Run the query
    content = types.Content(
        role="user",
        parts=[types.Part(text=query)]
    )

    final_response = ""
    async for event in runner.run_async(
        user_id="kaggle_user",
        session_id=session.id,
        new_message=content
    ):
        if event.is_final_response():
            if event.content and event.content.parts:
                final_response = event.content.parts[0].text

    return final_response


# Helper for Jupyter notebooks
def process_query(query: str) -> str:
    """Sync wrapper for run_infoshield."""
    return asyncio.get_event_loop().run_until_complete(run_infoshield(query))


print("✅ Runner ready!")

## 🧪 Test Queries

Let's test with different types of queries.

In [ ]:
# Test Query 1: High Urgency Emergency
query1 = "Help! There's flooding in Chennai, water is entering houses! Need immediate help!"

response = await run_infoshield(query1)
print(response)

In [ ]:
# Test Query 2: Information Request
query2 = "Is there a tsunami warning for the coast of Japan?"

response = await run_infoshield(query2)
print(response)

In [ ]:
# Test Query 3: General Preparedness
query3 = "What should I do to prepare for an earthquake?"

response = await run_infoshield(query3)
print(response)

In [ ]:
# Test Query 4: Low Credibility (should trigger human review)
query4 = "Aliens are attacking the city center!"

response = await run_infoshield(query4)
print(response)

## 📊 Interactive Mode

Try your own queries!

In [ ]:
# Interactive mode - enter your own query
user_query = input("Enter your disaster query: ")

if user_query.strip():
    response = await run_infoshield(user_query)
    print(response)
else:
    print("No query entered.")

## 📝 Human Review Queue

Check queries flagged for human expert review.

In [ ]:
# Display pending human reviews
if HUMAN_REVIEW_QUEUE:
    print(f"📋 Pending Human Reviews ({len(HUMAN_REVIEW_QUEUE)}):")
    print("-" * 60)
    for entry in HUMAN_REVIEW_QUEUE:
        print(f"  ID: {entry['session_id']}")
        print(f"  Query: {entry['query'][:50]}...")
        print(f"  Credibility: {entry['credibility_score']}/100")
        print(f"  Status: {entry['status']}")
        print()
else:
    print("✅ No pending human reviews.")

## 🎯 Summary

InfoShield AI demonstrates a **multi-agent architecture** using Google ADK:

1. **Analyzer Agent**: Extracts intent, urgency, and location
2. **Search Agent**: Performs real-time web searches
3. **Credibility Agent**: Scores information reliability
4. **Synthesizer Agent**: Creates the final report

### Key Features Demonstrated:
- ✅ Sequential agent orchestration with `SequentialAgent`
- ✅ State sharing between agents via `output_key`
- ✅ Built-in `google_search` tool integration
- ✅ Custom `FunctionTool` for business logic
- ✅ Human-in-the-loop for low-confidence queries

---

**⚠️ Disclaimer:** This AI system is for informational purposes only. For life-threatening emergencies, always contact local emergency services (911, 112, etc.) directly.